In [57]:
"Hello, my name is Allen.  I am ten years old.  I am tall, and my father is tall, too.  His name is David.  He is forty-five years old, and he is an office worker.  My mom, Cindy, is thirty-eight.  She is a housewife.  My grandma, Susan, is seventy years old.  She is a good cook.  We have a dog, too.  His name is Lucky.  We are a happy family.\
（　）(１)　Who is Cindy?　(Ａ)　A good cook.　(Ｂ)　Allen’s mom.　(Ｃ)　A housewife.　(Ｄ)　Allen’s grandma.\
（　）(２)　How old is David?　(Ａ)　10.　(Ｂ)　38.　(Ｃ)　45.　(Ｄ)　70.\
（　）(３)　Which（哪一個）is true（正確的）?　(Ａ)　Susan is a good cook.　(Ｂ)　Allen is a short boy.　(Ｃ)　Cindy is a writer.　(Ｄ)　David is a doctor.\
".strip()



'Hello, my name is Allen.  I am ten years old.  I am tall, and my father is tall, too.  His name is David.  He is forty-five years old, and he is an office worker.  My mom, Cindy, is thirty-eight.  She is a housewife.  My grandma, Susan, is seventy years old.  She is a good cook.  We have a dog, too.  His name is Lucky.  We are a happy family.（\u3000）(１)\u3000Who is Cindy?\u3000(Ａ)\u3000A good cook.\u3000(Ｂ)\u3000Allen’s mom.\u3000(Ｃ)\u3000A housewife.\u3000(Ｄ)\u3000Allen’s grandma.（\u3000）(２)\u3000How old is David?\u3000(Ａ)\u300010.\u3000(Ｂ)\u300038.\u3000(Ｃ)\u300045.\u3000(Ｄ)\u300070.（\u3000）(３)\u3000Which（哪一個）is true（正確的）?\u3000(Ａ)\u3000Susan is a good cook.\u3000(Ｂ)\u3000Allen is a short boy.\u3000(Ｃ)\u3000Cindy is a writer.\u3000(Ｄ)\u3000David is a doctor.'

In [229]:
from elasticsearch import Elasticsearch

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = "qz666"

# Create the client instance
es = Elasticsearch(
    "http://localhost:9200",
    basic_auth=("elastic", ELASTIC_PASSWORD),
    verify_certs=False,
    ssl_show_warn=False
)

# Successful response!
es.info()

ObjectApiResponse({'name': '10055f9184fe', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'MN00F2d8TourjrzCJz2PTw', 'version': {'number': '8.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a846182fa16b4ebfcc89aa3c11a11fd5adf3de04', 'build_date': '2022-11-17T18:56:17.538630285Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [260]:
import json

body = {
    "settings": {
        "number_of_shards": 3,
        "number_of_replicas": 2
    },
    "mappings": {
        "properties": {
            "index": {
                "type": "keyword"
            },
            "content": {
                "type": "text"
        }
    }
    }
}


es.indices.create(index='english', body=body)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'english'})

In [261]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import json
import re

def check(_id):
    query = {
        "query": {
            "bool": {
                "must": {
                    "term": {
                        "index": _id
                    }
                }
            }
        }
    }
    result = es.search(index='english', body=query)
    return result['hits']['total']['value']
    
result = es.search(index='english', body=query)
result['hits']['total']['value']    
    
with open('corpus.json', 'r')as f:
    data = f.read()
    corpus = eval(data)
    f.close()

def load_datas():
    actions = list()
    for idx in corpus.keys():
        flag = 0 # cloze
        for i, parag in enumerate(corpus[idx]):
            parag = parag.replace("'s", " is").replace("'re", " are")
            parag = re.sub(r'[\u0080-\uffef]', '', parag)
            parag = re.sub(r'[^a-zA-Z]', ' ', parag)
            if len(parag) == 0:
                flag = 1 # reading
                n_cloze = i
                continue
            if flag == 0: # cloze
                index = 'h-' + idx + '-c-' + str(i)
            else: # reading
                index = 'h-' + idx + '-r-' + str(i-n_cloze)
            if check(index) > 0:
                continue
            actions.append({
                "_index": "english",
                "_op_type": "index",
                "_source": {
                    "index": index,
                    "content": parag
                }
            })
    return actions


data = load_datas()
helpers.bulk(es, data)

/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


(35, [])

In [262]:
q1 = {"query": {
        "multi_match": {
            "query":    "Hi",
            "fields":   [ "content" ]
        }
    }
}

q2 = {"query": {
        "multi_match": {
            "query":    "h-1-1-c-1",
            "fields":   [ "index" ]
        }
    }
}

q3 = {"query": {
        "bool": {
            "must": {
                "term": {
                    "index": "h-1-1-c-1"
                }
            }
        }
    }
}

result = es.search(index='english', body=q3, size=100) # size is maximun number of hits
count = len(result['hits']['hits'])
print(count)

1


/Users/adam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


In [243]:
result['hits']['hits']

[{'_index': 'english',
  '_id': 'SBDTvoQBWLfKW9tYYBw_',
  '_score': 2.8526313,
  '_source': {'index': 'h-1-1-c-1',
   'content': 'Milly  Hi  Annie   Nice to see you again Annie  Hi  Milly   Nice to see you  too      Milly  Not bad   Thank you       you Annie  Im OK   Thanks Milly  Annie  this is my sister  Sue Annie  Hello  Sue      Sue  Nice to meet you  too     How do you do   Who are you   How are you   What are you     Oh  Really  Hows  And    Nice to meet you   Nice to see you   How are you   How do you do '}},
 {'_index': 'english',
  '_id': 'k4ULvoQBuWcRVMrxUc4g',
  '_score': 2.8526313,
  '_source': {'index': 'h-1-1-c-1',
   'content': 'Milly  Hi  Annie   Nice to see you again Annie  Hi  Milly   Nice to see you  too      Milly  Not bad   Thank you       you Annie  Im OK   Thanks Milly  Annie  this is my sister  Sue Annie  Hello  Sue      Sue  Nice to meet you  too     How do you do   Who are you   How are you   What are you     Oh  Really  Hows  And    Nice to meet you   Nice to

In [259]:
es.options(ignore_status=[400,404]).indices.delete(index='english')

ObjectApiResponse({'acknowledged': True})